In [115]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn.ensemble as ensemble

idata = pd.read_csv('data\\data.csv')
idata.shape
idata.isnull().sum()

POLICY_REF_DUMMY                0
PROPOSAL_SUBMISSION_DATE     5024
FIRST_UW_DEC                  336
LAST_UW_DEC                   336
ISSUE_DATE                    785
ST_DATE                       785
POL_ST                          0
QRY                             0
PRODUCT_CATEGORY                0
PROD_TYPE                       0
PRODUCT_CODE                    0
PRODUCT_NAME_L1              7945
PRODUCT_NAME                    0
BASIC_COVER_TYPE                0
COVER_CODE                      0
COVER_TYPE                      0
COVER_CAT                   20752
INSURED_AMT                   443
PREMIUM                      1112
COVER_TERM                    962
PAY_TERM                      962
PAY_METHOD                      0
PAY_MODE                        0
PAY_AGE                       962
COVER_AGE                     962
INSURED_ID                      0
BUYER_ID                    12617
RELATIONSHIP                    0
RELATIONSHIP_BIN                0
ENTRY_AGE     

In [116]:
# Drop columns with complete NaN value

idata.drop(['COVER_CAT','SMOKER_STATUS','SMK_NO_CIG', 'AGENCY', 'CHANNEL'], axis=1, inplace=True)

In [117]:
# Remove 67 duplicated records

idata.drop_duplicates(inplace=True)

In [118]:
#based on the condition RELATIONSHIP=='SELF, fill column BUYER_ID by INSURED_ID

idata.loc[idata.RELATIONSHIP=='SELF','BUYER_ID'] = idata['BUYER_ID'][idata.RELATIONSHIP=='SELF'].fillna(idata.INSURED_ID)

In [119]:
#fill ISSUE_DATE & ST_DATE by PROPOSAL_SUBMISSION_DATE

idata.ISSUE_DATE.fillna(idata.PROPOSAL_SUBMISSION_DATE, inplace=True)
idata.ST_DATE.fillna(idata.PROPOSAL_SUBMISSION_DATE, inplace=True)

In [120]:
#generate new column OWNER_REAL_AGE,INSURED_AGE since owner and isured age is increasing as time passes by.
#using 2020 to calculate ANB

idata['BIRTH_YEAR'] = pd.to_datetime(idata.ISSUE_DATE).dt.year - idata.OWNER_AGE
idata['OWNER_REAL_AGE'] = 2020 - idata.BIRTH_YEAR

idata['INSURED_BIRTH_YEAR'] = pd.to_datetime(idata.ISSUE_DATE).dt.year - idata.ENTRY_AGE
idata['INSURED_AGE'] = 2020 - idata.INSURED_BIRTH_YEAR


# idata.loc[idata.RELATIONSHIP == 'SELF','OWNER_REAL_AGE'].fillna(idata.loc[idata.RELATIONSHIP == 'SELF','INSURED_AGE'], inplace=True)
# idata.loc[idata.RELATIONSHIP == 'SELF','OWNER_REAL_AGE'].fillna(idata.loc[idata.RELATIONSHIP == 'SELF','OWNER_AGE'], inplace=True)
# idata.loc[idata.RELATIONSHIP == 'SELF','INSURED_AGE'].fillna(idata.loc[idata.RELATIONSHIP == 'SELF','OWNER_AGE'], inplace=True)

In [121]:
#create column ANP

idata['ANP'] = np.where(idata['PAY_METHOD']=='Single Pay', idata['PREMIUM'].str.replace(',', '').astype(float)/10, idata['PREMIUM'].str.replace(',', '').astype(float))
idata.head()

,POLICY_REF_DUMMY,PROPOSAL_SUBMISSION_DATE,FIRST_UW_DEC,LAST_UW_DEC,ISSUE_DATE,ST_DATE,POL_ST,QRY,PRODUCT_CATEGORY,PROD_TYPE,...,OWNER_OCCUPATION,OWNER_OCCUPATION_CLASS,CLASS_ORDER,SERV_BRANCH_DUMMY,AGENT_CD_DUMMY,BIRTH_YEAR,OWNER_REAL_AGE,INSURED_BIRTH_YEAR,INSURED_AGE,ANP
0,POLID_6930,NaN,Issued Policy (UND),OK TO ISSUE,03/19/2013,04/23/2014,SURRENDERED,UL_BASIC,Investment,UL,...,PLANNER,NaN,0,SERV_BRANCH_398,NaN,1979.0,41.0,1979.0,41.0,10000.0
1,POLID_3215,06/30/2011,Tagged as OK to issue,OK TO ISSUE,06/15/2011,06/08/2011,Inforce,TRAD_BASIC,Retirement,NUL,...,FAMILY MEDICINE - DOCTOR,"HOSPITAL, NURSING HOME, CLINIC, ASSYLUM",4,SERV_BRANCH_452,AGENT_CD_58,1958.0,62.0,1958.0,62.0,161515.0
2,POLID_7224,NaN,Issued Policy (UND),OK TO ISSUE,06/18/2013,06/17/2013,INFORCE,UL_BASIC,Investment,UL,...,HOUSEWIFE,NaN,0,SERV_BRANCH_564,AGENT_CD_42,1954.0,66.0,1954.0,66.0,45000.0
3,POLID_54,NaN,Tagged as OK to issue,OK TO ISSUE,12/14/2010,12/13/2010,Inforce,TRAD_BASIC,Retirement,NUL,...,BANK MANAGER,BANK,10,SERV_BRANCH_600,AGENT_CD_41,1960.0,60.0,1991.0,29.0,26745.0
4,POLID_54,NaN,Tagged as OK to issue,OK TO ISSUE,12/14/2010,12/13/2010,Inforce,TRAD_RIDER,Protection,NUL,...,BANK MANAGER,BANK,10,SERV_BRANCH_600,AGENT_CD_41,1960.0,60.0,1991.0,29.0,443.0


In [122]:
#transform column medical_flag

idata['NEW_MEDICAL_FLAG'] = idata.MEDICAL_FLAG.apply(lambda x: 1.0 if x=='Y' else 0.0)
idata.drop(['MEDICAL_FLAG'], axis=1, inplace=True)
              

In [123]:
# Transform Insured Height
def transformHeight(x):
    if isinstance(x, float) != True:
        y = x[len(x)-2 : len(x)]
        if y == 'FI':
            f_list = x.split(',')
            for i in range(len(f_list)-1):
                if f_list[i] == "":
                    f_list[i] = 0
                else:
                    f_list[i] = float(f_list[i])
            return (f_list[0] * 30.48 + f_list[1] * 2.54)/100
        if y == 'CM':
            c_list = x.split(',')
            for i in range(len(c_list)-1):
                if c_list[i] == "":
                    c_list[i] = 0
                else:
                    c_list[i] = float(c_list[i])
            return c_list[0]/100
    
    
idata['HEIGHT'] = idata.HEIGHT.apply(transformHeight)

In [124]:
# Transform Insured Weight
def transformWeight(x):
    if isinstance(x, float) != True:
        y = x[len(x)-1 : len(x)]
        if y == 'P':
            p_weight = float(x[0:len(x)-1]) * 0.45
            return p_weight
        if y == 'K':
            k_weight = float(x[0:len(x)-1])
            return k_weight
    
    
idata['WEIGHT'] = idata.WEIGHT.apply(transformWeight)

In [125]:
# fill AGENT_CD_DUMMY missing data

newdata = idata.groupby(['SERV_BRANCH_DUMMY']).AGENT_CD_DUMMY.value_counts()
# print(newdata.SERV_BRANCH_1.index)
newdata.index.levels[0]
# print(newdata.SERV_BRANCH_1.index[0])
newdict = {}
for idx in newdata.index.levels[0]:
    if idx in newdata.index:
        if len(newdata[idx].index)!=0:
            newdict[idx] = newdata[idx].index[0]
            
for i in idata.index:
    if pd.isna(idata.AGENT_CD_DUMMY[i]):
        serv_name = idata.SERV_BRANCH_DUMMY[i]
        idata.AGENT_CD_DUMMY.at[i] = newdict.get(serv_name,'AGENT_CD_14')

In [126]:
#fillna most_frequent

from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer( strategy='most_frequent')

imp_mean.fit(idata['ST_DATE'].values.reshape(-1,1))
idata['ST_DATE'] = imp_mean.transform(idata['ST_DATE'].values.reshape(-1,1))

imp_mean.fit(idata['OCCUPATION'].values.reshape(-1,1))
idata['OCCUPATION'] = imp_mean.transform(idata['OCCUPATION'].values.reshape(-1,1))

imp_mean.fit(idata['OCCUPATION_CLASS'].values.reshape(-1,1))
idata['OCCUPATION_CLASS'] = imp_mean.transform(idata['OCCUPATION_CLASS'].values.reshape(-1,1))

imp_mean.fit(idata['NATIONALITY_DUMMY'].values.reshape(-1,1))
idata['NATIONALITY_DUMMY'] = imp_mean.transform(idata['NATIONALITY_DUMMY'].values.reshape(-1,1))

imp_mean.fit(idata['REGION_DUMMY'].values.reshape(-1,1))
idata['REGION_DUMMY'] = imp_mean.transform(idata['REGION_DUMMY'].values.reshape(-1,1))

imp_mean.fit(idata['OWNER_OCCUPATION'].values.reshape(-1,1))
idata['OWNER_OCCUPATION'] = imp_mean.transform(idata['OWNER_OCCUPATION'].values.reshape(-1,1))

imp_mean.fit(idata['OWNER_OCCUPATION_CLASS'].values.reshape(-1,1))
idata['OWNER_OCCUPATION_CLASS'] = imp_mean.transform(idata['OWNER_OCCUPATION_CLASS'].values.reshape(-1,1))

idata.isnull().sum()

POLICY_REF_DUMMY               0
PROPOSAL_SUBMISSION_DATE    4991
FIRST_UW_DEC                 336
LAST_UW_DEC                  336
ISSUE_DATE                    59
ST_DATE                        0
POL_ST                         0
QRY                            0
PRODUCT_CATEGORY               0
PROD_TYPE                      0
PRODUCT_CODE                   0
PRODUCT_NAME_L1             7878
PRODUCT_NAME                   0
BASIC_COVER_TYPE               0
COVER_CODE                     0
COVER_TYPE                     0
INSURED_AMT                  443
PREMIUM                     1112
COVER_TERM                   962
PAY_TERM                     962
PAY_METHOD                     0
PAY_MODE                       0
PAY_AGE                      962
COVER_AGE                    962
INSURED_ID                     0
BUYER_ID                       0
RELATIONSHIP                   0
RELATIONSHIP_BIN               0
ENTRY_AGE                      0
GENDER                         0
OCCUPATION

In [127]:
#fillna median

imp_mean = SimpleImputer( strategy='median')

imp_mean.fit(idata['OWNER_REAL_AGE'].values.reshape(-1,1))
idata['OWNER_REAL_AGE'] = imp_mean.transform(idata['OWNER_REAL_AGE'].values.reshape(-1,1))

imp_mean.fit(idata['OWNER_AGE'].values.reshape(-1,1))
idata['OWNER_AGE'] = imp_mean.transform(idata['OWNER_AGE'].values.reshape(-1,1))

imp_mean.fit(idata['INSURED_AGE'].values.reshape(-1,1))
idata['INSURED_AGE'] = imp_mean.transform(idata['INSURED_AGE'].values.reshape(-1,1))

imp_mean.fit(idata['ANP'].values.reshape(-1,1))
idata['ANP'] = imp_mean.transform(idata['ANP'].values.reshape(-1,1))

imp_mean.fit(idata['WEIGHT'].values.reshape(-1,1))
idata['WEIGHT'] = imp_mean.transform(idata['WEIGHT'].values.reshape(-1,1))

imp_mean.fit(idata['HEIGHT'].values.reshape(-1,1))
idata['HEIGHT'] = imp_mean.transform(idata['HEIGHT'].values.reshape(-1,1))

idata.isnull().sum()

POLICY_REF_DUMMY               0
PROPOSAL_SUBMISSION_DATE    4991
FIRST_UW_DEC                 336
LAST_UW_DEC                  336
ISSUE_DATE                    59
ST_DATE                        0
POL_ST                         0
QRY                            0
PRODUCT_CATEGORY               0
PROD_TYPE                      0
PRODUCT_CODE                   0
PRODUCT_NAME_L1             7878
PRODUCT_NAME                   0
BASIC_COVER_TYPE               0
COVER_CODE                     0
COVER_TYPE                     0
INSURED_AMT                  443
PREMIUM                     1112
COVER_TERM                   962
PAY_TERM                     962
PAY_METHOD                     0
PAY_MODE                       0
PAY_AGE                      962
COVER_AGE                    962
INSURED_ID                     0
BUYER_ID                       0
RELATIONSHIP                   0
RELATIONSHIP_BIN               0
ENTRY_AGE                      0
GENDER                         0
OCCUPATION

In [128]:
# Create column BMI
idata['BMI'] = idata.WEIGHT / np.square(idata.HEIGHT)

In [129]:
#delete deemed prediction-irrelevant columns

idata.drop(['POLICY_REF_DUMMY','PROPOSAL_SUBMISSION_DATE','FIRST_UW_DEC', 'LAST_UW_DEC', 'ISSUE_DATE','POL_ST' ,'QRY', 
                'PROD_TYPE', 'PRODUCT_CODE', 'PRODUCT_NAME_L1', 'PRODUCT_NAME','BASIC_COVER_TYPE', 'INSURED_AMT' ,'PREMIUM','COVER_TERM', 
                'PAY_TERM','PAY_METHOD', 'PAY_MODE','PAY_AGE', 'COVER_AGE' ,'HEIGHT','WEIGHT', 'BIRTH_YEAR', 
                'INSURED_BIRTH_YEAR'], axis=1, inplace=True)
    
idata.shape

(20685, 26)

In [130]:
#Label encoding & one-hot encoding

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

idata['PRODUCT_CATEGORY'] = le.fit_transform(idata['PRODUCT_CATEGORY'])
idata['BUYER_ID'] = le.fit_transform(idata['BUYER_ID'])
idata['COVER_TYPE'] = le.fit_transform(idata['COVER_TYPE'])
idata['INSURED_ID'] = le.fit_transform(idata['INSURED_ID'])
idata['RELATIONSHIP'] = le.fit_transform(idata['RELATIONSHIP'])
idata['RELATIONSHIP_BIN'] = le.fit_transform(idata['RELATIONSHIP_BIN'])
idata['OCCUPATION'] = le.fit_transform(idata['OCCUPATION'])
idata['OCCUPATION_CLASS'] = le.fit_transform(idata['OCCUPATION_CLASS'])
idata['NATIONALITY_DUMMY'] = le.fit_transform(idata['NATIONALITY_DUMMY'])
idata['REGION_DUMMY'] = le.fit_transform(idata['REGION_DUMMY'])
idata['OWNER_OCCUPATION'] = le.fit_transform(idata['OWNER_OCCUPATION'])
idata['OWNER_OCCUPATION_CLASS'] = le.fit_transform(idata['OWNER_OCCUPATION_CLASS'])
idata['SERV_BRANCH_DUMMY'] = le.fit_transform(idata['SERV_BRANCH_DUMMY'])
idata['AGENT_CD_DUMMY'] = le.fit_transform(idata['AGENT_CD_DUMMY'])
idata['GENDER'] = le.fit_transform(idata['GENDER'])

idata = pd.concat([idata,pd.get_dummies(idata['OWNER_GENDER'], prefix='OWNER_GENDER',dummy_na=True)],axis=1).drop(['OWNER_GENDER'],axis=1)

idata.head()

,ST_DATE,PRODUCT_CATEGORY,COVER_CODE,COVER_TYPE,INSURED_ID,BUYER_ID,RELATIONSHIP,RELATIONSHIP_BIN,ENTRY_AGE,GENDER,...,SERV_BRANCH_DUMMY,AGENT_CD_DUMMY,OWNER_REAL_AGE,INSURED_AGE,ANP,NEW_MEDICAL_FLAG,BMI,OWNER_GENDER_F,OWNER_GENDER_M,OWNER_GENDER_nan
0,04/23/2014,1,"2,161",0,758,559,24,8,34,0,...,331,28,41.0,41.0,10000.0,0.0,21.556748,1,0,0
1,06/08/2011,3,"2,198",0,10402,10274,24,8,53,0,...,392,311,62.0,62.0,161515.0,0.0,21.774237,1,0,0
2,06/17/2013,1,"2,163",0,5479,4229,24,8,59,0,...,516,294,66.0,66.0,45000.0,0.0,25.403277,1,0,0
3,12/13/2010,3,"2,127",0,7827,8527,5,0,19,0,...,557,293,60.0,29.0,26745.0,0.0,19.375039,1,0,0
4,12/13/2010,2,"1,476",1,7827,8527,5,0,19,0,...,557,293,60.0,29.0,443.0,0.0,19.375039,1,0,0


In [131]:
#creating trainset &testset

idata['ST_DATE'] = pd.to_datetime(idata['ST_DATE'])
idx = idata.groupby('BUYER_ID')['ST_DATE'].transform(max)==idata['ST_DATE']
tdata = idata[idx].groupby('BUYER_ID').first()
tdata.reset_index('BUYER_ID',inplace=True)


idata.drop(['ST_DATE','OWNER_REAL_AGE', 'INSURED_AGE', 'COVER_CODE','ANP'], axis=1, inplace=True)
tdata.drop(['ST_DATE','ENTRY_AGE', 'OWNER_AGE', 'PRODUCT_CATEGORY', 'COVER_CODE','ANP'], axis=1, inplace=True)


#replace OWNER_AGE & ENTRY_AGE with current age of Owner and Insured 
tdata.rename(columns={"OWNER_REAL_AGE": "OWNER_AGE", "INSURED_AGE":"ENTRY_AGE"},inplace=True)
                    
train_x,train_y = idata.iloc[:,1:],idata.iloc[:,0]
test_x = tdata

train_x.sort_index(axis = 1,inplace = True)
test_x.sort_index(axis = 1,inplace = True)

In [148]:
#export 

train_x.to_csv('x_train.csv',index=False)
train_y.to_csv('y_train.csv',index=False)
test_x.to_csv('x_test.csv',index=False)

C:\Users\SONY\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  after removing the cwd from sys.path.


In [188]:
#run grid searchCV to determine the best combination of hyperparameters

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV 
from sklearn.model_selection import train_test_split 
from sklearn import neighbors


model = ensemble.RandomForestClassifier()           

n_estimators = [100, 200, 300]
max_depth = [15, 25, 50]
class_wight =  [{0:1,1:72,2:95,3:45,4:12}]
#class_wight = [{0: w} for w in [1, 72, 95, 45, 12]]

param_grid = dict(n_estimators = n_estimators, max_depth = max_depth, class_weight = class_wight)
 
kflod = StratifiedKFold(n_splits=10, shuffle = True,random_state=7)

grid_search = GridSearchCV(model,param_grid,scoring = 'accuracy',n_jobs = -1,cv = kflod)

grid_result = grid_search.fit(train_x, train_y) 
print("Best: %f using %s" % (grid_result.best_score_,grid_search.best_params_))

means = grid_result.cv_results_['mean_test_score']
params = grid_result.cv_results_['params']
for mean,param in zip(means,params):
    print("%f  with:   %r" % (mean,param))

Best: 0.769011 using {'class_weight': {0: 1, 1: 72, 2: 95, 3: 45, 4: 12}, 'max_depth': 15, 'n_estimators': 200}
0.767029  with:   {'class_weight': {0: 1, 1: 72, 2: 95, 3: 45, 4: 12}, 'max_depth': 15, 'n_estimators': 100}
0.769011  with:   {'class_weight': {0: 1, 1: 72, 2: 95, 3: 45, 4: 12}, 'max_depth': 15, 'n_estimators': 200}
0.768528  with:   {'class_weight': {0: 1, 1: 72, 2: 95, 3: 45, 4: 12}, 'max_depth': 15, 'n_estimators': 300}
0.762243  with:   {'class_weight': {0: 1, 1: 72, 2: 95, 3: 45, 4: 12}, 'max_depth': 25, 'n_estimators': 100}
0.763065  with:   {'class_weight': {0: 1, 1: 72, 2: 95, 3: 45, 4: 12}, 'max_depth': 25, 'n_estimators': 200}
0.763452  with:   {'class_weight': {0: 1, 1: 72, 2: 95, 3: 45, 4: 12}, 'max_depth': 25, 'n_estimators': 300}
0.761083  with:   {'class_weight': {0: 1, 1: 72, 2: 95, 3: 45, 4: 12}, 'max_depth': 50, 'n_estimators': 100}
0.763065  with:   {'class_weight': {0: 1, 1: 72, 2: 95, 3: 45, 4: 12}, 'max_depth': 50, 'n_estimators': 200}
0.763307  with: 

In [189]:
#run random forest and display feature_importances

RFR = ensemble.RandomForestClassifier(n_estimators=200,max_depth=15, class_weight = {0:1,1:72,2:95,3:45,4:12})
RFR.fit(train_x,train_y)
prediction_RFR = RFR.predict(test_x)
RFR.feature_importances_

array([0.02798893, 0.05880677, 0.0330287 , 0.03885205, 0.50255873,
       0.07291641, 0.00399077, 0.03343335, 0.00186438, 0.0010471 ,
       0.02710689, 0.02204557, 0.05983992, 0.00329664, 0.00341386,
       0.00070976, 0.02203025, 0.01666797, 0.01208052, 0.01224263,
       0.01629893, 0.02977987])

In [190]:
def thredhold(x):
    if x == 0:
        return 'Health'
    elif x == 1:
        return 'Investment'
    elif x == 2:
        return 'Protection'
    elif x == 3:
        return 'Retirement'
    elif x == 4:
        return 'Savings'
    else:
        return None
    
pdata = pd.DataFrame(prediction_RFR, columns=['predictions'])
pdata = pdata.predictions.apply(thredhold)
# pdata.head()

In [191]:
#export prediction

pdata.to_csv('prediction_RFR.csv')

C:\Users\SONY\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  This is separate from the ipykernel package so we can avoid doing imports until
